<a href="https://colab.research.google.com/github/VictorSairam/House_Price_Prediction/blob/main/house_price_prediction_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
 import numpy as np
 import tensorflow as tf
 from tensorflow import keras
 import pandas as pd
 import matplotlib.pyplot as plt
 %matplotlib inline
 from google.colab import files
 data_to_load = files.upload()

Saving insurance_data.csv to insurance_data (1).csv


In [5]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size = 0.2, random_state = 25)

In [7]:
len(x_train)

22

In [8]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [9]:
x_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [10]:
 model = keras.Sequential([
     keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
 ])

 model.compile(
     optimizer = 'adam',
     loss = 'binary_crossentropy',
     metrics = ['accuracy']
 )

 model.fit(x_train_scaled, y_train, epochs = 1000)

Epoch 1/1000
1/1 [==============================] - 0s 421ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/1000
1/1

In [11]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 0s 424ms/step - loss: 0.5855 - accuracy: 0.6667


[0.5854933857917786, 0.6666666865348816]

In [12]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 139ms/step


array([[0.64471424],
       [0.5669487 ],
       [0.40433866],
       [0.5943528 ],
       [0.64985454],
       [0.6799391 ]], dtype=float32)

In [13]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [14]:
coef, intercept = model.get_weights()
coef, intercept

(array([[1.1257654],
        [0.7468934]], dtype=float32), array([-0.6801185], dtype=float32))

In [15]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

In [16]:
sigmoid(18)

0.9999999847700205

In [17]:
def prediction_function(age, affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability + intercept
  return sigmoid(weighted_sum)

In [18]:
prediction_function(0.18, 1)

0.5669487171010201

In [19]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  log_loss = -np.mean(y_true*np.log(y_predicted_new) + (1 - y_true)*np.log(1 - y_predicted_new))
  return log_loss

In [20]:
def sigmoid_numpy(X):
  return 1 / (1 + np.exp(-X))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [21]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
  # w1, w2, w3
  w1 = w2 = 1
  bias = 0
  rate = 0.5
  n = len(age)

  for i in range(epochs):
    weighted_sum = w1*age + w2*affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true, y_predicted)

    w1d = (1/n)*(np.dot(np.transpose(age),(y_predicted - y_true)))
    w2d = (1/n)*(np.dot(np.transpose(affordability),(y_predicted - y_true)))

    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias - rate * bias_d
    if loss <= loss_threshold:
      break


  return w1, w2, bias, loss

In [22]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordibility'], y_train, 1000, 0.4631)

(5.051047623653049,
 1.4569794548473887,
 -2.9596534546250037,
 0.46293944095888917)

In [36]:
class myNN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def fit(self, x, y, epochs, loss_threshold):
    self.w1, self.w2, self.bias = self.gradient_descent(x['age'], x['affordibility'], y, epochs, loss_threshold)
  
  def predict(self, x_test):
    weighted_sum = self.w1 * x_test['age'] + self.w2 * x_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)

  def gradient_descent(self, age, affordability, y_true, epochs, loss_threshold):
  # w1, w2, w3
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
      weighted_sum = w1*age + w2*affordability + bias
      y_predicted = sigmoid_numpy(weighted_sum)
      loss = log_loss(y_true, y_predicted)

      w1d = (1/n)*(np.dot(np.transpose(age),(y_predicted - y_true)))
      w2d = (1/n)*(np.dot(np.transpose(affordability),(y_predicted - y_true)))

      bias_d = np.mean(y_predicted - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * bias_d
      if loss <= loss_threshold:
        break
      # if i%50 == 0:
      print("Loss:", w1)

    return w1, w2, bias

In [37]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs=1000, loss_threshold=0.4631)

Loss: 0.974907633470177
Loss: 0.9556229728273669
Loss: 0.9416488476693794
Loss: 0.9323916996249162
Loss: 0.9272267472726993
Loss: 0.9255469396815343
Loss: 0.9267936114129968
Loss: 0.93047170420295
Loss: 0.9361540784567942
Loss: 0.9434791243557357
Loss: 0.9521448361628082
Loss: 0.9619014360798376
Loss: 0.9725437902239876
Loss: 0.9839042828641819
Loss: 0.9958464546516588
Loss: 1.0082595007242081
Loss: 1.0210536111133566
Loss: 1.034156080666481
Loss: 1.0475080939682688
Loss: 1.0610620872868053
Loss: 1.0747795953648005
Loss: 1.0886295007650375
Loss: 1.1025866146156011
Loss: 1.1166305284975315
Loss: 1.130744687166233
Loss: 1.1449156405234902
Loss: 1.1591324407175467
Loss: 1.1733861565198167
Loss: 1.1876694823356402
Loss: 1.2019764234961199
Loss: 1.2163020429886013
Loss: 1.2306422576428844
Loss: 1.244993674111329
Loss: 1.2593534568600255
Loss: 1.2737192219062514
Loss: 1.2880889512619975
Loss: 1.3024609240300042
Loss: 1.3168336608930384
Loss: 1.3312058793761508
Loss: 1.3455764577755067
Loss: 

In [31]:
coef, intercept

(array([[1.1257654],
        [0.7468934]], dtype=float32), array([-0.6801185], dtype=float32))

In [39]:
customModel.predict(x_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

In [41]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 13ms/step


array([[0.64471424],
       [0.5669487 ],
       [0.40433866],
       [0.5943528 ],
       [0.64985454],
       [0.6799391 ]], dtype=float32)